In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("jjzha/jobbert_skill_extraction")
model = AutoModelForTokenClassification.from_pretrained(
    "jjzha/jobbert_skill_extraction"
)

d:\code\automate-resume\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('baseline_taxonomies/skills.txt') as f:
    skills = f.read().splitlines()

with open('baseline_taxonomies/base_roles.txt') as f:
    base_roles = f.read().splitlines()

In [ ]:
def get_embeddings(s):
    inputs = tokenizer(s, return_tensors='pt', padding=True, truncation=True)
    outputs = model(**inputs)
    hidden_states = outputs.hidden_states
        
    last_four_layers = hidden_states[-4:]
    embedding = torch.stack(last_four_layers).mean(dim=0).mean(dim=1)

    # return outputs.mean(dim=1)
    return embedding.detach().numpy()


In [20]:
# Generate embeddings for each skill and field
# skill_embeddings = [get_embeddings(skill) for skill in skills]
field_embeddings = [get_embeddings(field) for field in base_roles]

In [21]:
print(field_embeddings[0])

[[[[-2.08777636e-02  4.41608094e-02  1.73158050e-02 ... -1.20266471e-02
     3.81970033e-02 -1.05189234e-01]
   [ 8.42915103e-02 -1.33205399e-01  3.82551461e-01 ...  1.31955817e-01
    -2.78771758e-01  6.56100273e-01]
   [ 1.07448228e-01 -9.55503404e-01  2.50813723e-01 ... -5.67837596e-01
    -6.83784842e-01 -4.41183150e-01]
   [-2.17896178e-02  4.25617471e-02  1.64222009e-02 ... -1.31094065e-02
     3.79499123e-02 -1.05652831e-01]]]


 [[[-4.84549329e-02  3.02449726e-02 -2.21534111e-02 ...  8.24693590e-04
     3.35202739e-03 -6.15342073e-02]
   [ 6.37015477e-02 -1.54448897e-01  1.42216638e-01 ... -2.87963320e-02
     2.43711993e-01  8.80814433e-01]
   [-4.53498960e-03 -8.65606368e-01  2.25552589e-01 ... -1.01776302e-01
    -6.47820354e-01 -1.88810930e-01]
   [-4.84745800e-02  2.99538150e-02 -2.20037028e-02 ...  7.72904605e-04
     3.28678638e-03 -6.16356768e-02]]]


 [[[-4.89330068e-02  2.67118048e-02 -2.59691849e-02 ...  5.24685159e-03
     3.40168551e-02 -6.58987463e-03]
   [-5.9948

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Compute similarity matrix
similarity_matrix = cosine_similarity(
    np.vstack(skill_embeddings), np.vstack(field_embeddings)
)

TypeError: float() argument must be a string or a real number, not 'TokenClassifierOutput'

In [ ]:
# plot heatmap for similarity matrix
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(similarity_matrix, xticklabels=base_roles, yticklabels=skills)